## 1) Configure paths and column mappings

In [26]:
# === User-configurable paths ===
from pathlib import Path
import os

# Input performance CSV (same folder by default)
PERF_FILE = "Performance - correctness_time.csv"

# Optional: perceived/practice and perceived/test CSVs (set to None if not used)
PERCEP_PRACTICE = None
PERCEP_TEST = None

# Output directory
OUTDIR = "outputs"
os.makedirs(OUTDIR, exist_ok=True)

# === Column name mapping ===
# Map your CSV's column names to the analysis' expected semantic names.
# Keys expected by the loader: participant_id, group, session, correctness, time_sec (as available/derivable).
# The provided sample CSV uses 'Participant_ID' and 'Group_ID'. Sessions are spread across columns (T0, P1, T1, ...).
MAPS = {
    # Map SOURCE column names in your CSV -> expected names used by analysis
    "Participant_ID": "participant_id",
    "Group_ID": "group",
    # If your file already has tidy columns, you can leave MAPS = {}
}


print('Performance file:', Path(PERF_FILE).resolve())
print('OUTDIR:', Path(OUTDIR).resolve())
print('Perceived (practice/test):', PERCEP_PRACTICE, PERCEP_TEST)
print('Column maps:', MAPS)

Performance file: /Users/zhangziyi/Projects/2025-Summer-TOCE-Resubmit-Instruction/Performance - correctness_time.csv
OUTDIR: /Users/zhangziyi/Projects/2025-Summer-TOCE-Resubmit-Instruction/outputs
Perceived (practice/test): None None
Column maps: {'Participant_ID': 'participant_id', 'Group_ID': 'group'}


## 2) Preview input format

In [27]:
import pandas as pd
df_preview = pd.read_csv(PERF_FILE)
display(df_preview.head(12))
print('Shape:', df_preview.shape)
print('Columns:', list(df_preview.columns))

,Metrics,Participant_ID,Group_ID,T0,P1,T1,P2,T2,P3,T3,P4,T4,P5,T5
0,Correctness,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Correctness,2,2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2,Correctness,3,3,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,Correctness,4,4,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
4,Correctness,5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
5,Correctness,6,2,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
6,Correctness,7,3,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
7,Correctness,8,4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
8,Correctness,9,1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
9,Correctness,10,2,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


Shape: (86, 14)
Columns: ['Metrics', 'Participant_ID', 'Group_ID', 'T0', 'P1', 'T1', 'P2', 'T2', 'P3', 'T3', 'P4', 'T4', 'P5', 'T5']


## 3) Load analysis functions

In [28]:
import argparse
import os
import math
import numpy as np
import pandas as pd
from itertools import combinations
from collections import defaultdict

# Statsmodels & SciPy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

In [29]:
# ------------------------ Utilities ------------------------

GROUP_LABELS = {
    "G1": "G1 = No Instruction (Control)",
    "G2": "G2 = Abstract Guidelines",
    "G3": "G3 = Concrete Steps (Context-Agnostic)",
    "G4": "G4 = Context-Specific Worked Examples",
}

def normalize_group(g):
    if isinstance(g, str):
        g = g.strip().upper()
    return g

def normalize_session(s):
    """Return numeric session order. Accepts 'T0', 'T1'.. or ints/strings."""
    if isinstance(s, str):
        s = s.strip().upper()
        if s.startswith('T'):
            try:
                return int(s[1:])
            except:
                pass
        try:
            return int(s)
        except:
            return None
    if isinstance(s, (int, np.integer, float)) and not np.isnan(s):
        return int(s)
    return None

def holm_bonferroni(pvals, labels, alpha=0.05):
    """Holm-Bonferroni adjustment. Returns DataFrame with adjusted decisions."""
    df = pd.DataFrame({"label": labels, "p": pvals})
    df = df.sort_values("p").reset_index(drop=True)
    m = len(df)
    adj = []
    reject = []
    for i, p in enumerate(df["p"], start=1):
        threshold = alpha / (m - i + 1)
        adj.append(threshold)
        reject.append(p <= threshold)
    df["holm_threshold"] = adj
    df["reject_at_alpha"] = reject
    return df.sort_index()

def ci_mean(x, alpha=0.05):
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]
    n = len(x)
    if n == 0:
        return (np.nan, np.nan, np.nan)
    m = np.mean(x)
    se = stats.sem(x, nan_policy='omit')
    if n > 1:
        tcrit = stats.t.ppf(1 - alpha/2, df=n-1)
        lo, hi = m - tcrit*se, m + tcrit*se
    else:
        lo, hi = np.nan, np.nan
    return (m, lo, hi)

def ci_prop(p_hat, n, alpha=0.05):
    """Wald CI with continuity correction; robust enough for reporting (small-N caution)."""
    if n == 0:
        return (np.nan, np.nan, np.nan)
    se = math.sqrt(p_hat * (1 - p_hat) / n)
    z = stats.norm.ppf(1 - alpha/2)
    lo, hi = p_hat - z * se, p_hat + z * se
    lo, hi = max(0, lo), min(1, hi)
    return (p_hat, lo, hi)

def bootstrap_ci(x, stat_func=np.median, B=2000, alpha=0.05, bca=True):
    """
    Bootstrap CI for a statistic (median by default).
    If bca=True, approximate BCa using jackknife acceleration.
    """
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]
    n = len(x)
    if n == 0:
        return (np.nan, np.nan, np.nan)
    stat0 = stat_func(x)
    rng = np.random.default_rng(42)
    idx = rng.integers(0, n, size=(B, n))
    boots = np.array([stat_func(x[i]) for i in idx])

    if not bca:
        lo = np.quantile(boots, alpha/2)
        hi = np.quantile(boots, 1 - alpha/2)
        return (stat0, lo, hi)

    # BCa
    z0 = stats.norm.ppf((boots < stat0).mean() + 1e-12)
    # jackknife
    jack = np.array([stat_func(np.delete(x, i)) for i in range(n)])
    jack_mean = jack.mean()
    num = np.sum((jack_mean - jack) ** 3)
    den = 6.0 * (np.sum((jack_mean - jack) ** 2) ** 1.5 + 1e-12)
    a = num / (den + 1e-12)

    def bca_quantile(q):
        z = stats.norm.ppf(q)
        adj = stats.norm.cdf(z0 + (z0 + z) / (1 - a * (z0 + z)))
        return adj

    ql = bca_quantile(alpha/2)
    qh = bca_quantile(1 - alpha/2)
    lo = np.quantile(boots, ql)
    hi = np.quantile(boots, qh)
    return (stat0, lo, hi)

def mad_outliers(x, thresh=3.0):
    """Return mask of values flagged as outliers under MAD rule ±thresh*MAD around median."""
    x = np.asarray(x, dtype=float)
    med = np.nanmedian(x)
    mad = np.nanmedian(np.abs(x - med)) + 1e-12
    dev = np.abs(x - med) / mad
    return dev > thresh

def iqr_outliers(x, k=1.5):
    x = np.asarray(x, dtype=float)
    q1, q3 = np.nanpercentile(x, [25, 75])
    iqr = q3 - q1
    lo = q1 - k * iqr
    hi = q3 + k * iqr
    return (x < lo) | (x > hi)

def ensure_numeric(v):
    try:
        return pd.to_numeric(v)
    except:
        return v

def expand_group_labels(series):
    return series.map(lambda g: GROUP_LABELS.get(str(g).upper(), str(g))).astype(str)


In [30]:

# ------------------------ Core pipeline ------------------------

def load_and_prepare(perf_path, maps, percep_practice=None, percep_test=None):
    df = pd.read_csv(perf_path)

    # Rename columns if mapping provided
    df = df.rename(columns=maps)

    # Normalize core columns
    for col in ["participant_id", "group", "session", "phase", "snippet_id"]:
        if col not in df.columns:
            continue
        if col == "session":
            df[col] = df[col].apply(normalize_session)
        elif col == "group":
            df[col] = df[col].apply(normalize_group)
        else:
            df[col] = df[col]

    # Correctness must be 0/1
    if "correctness" in df.columns:
        df["correctness"] = pd.to_numeric(df["correctness"], errors="coerce").astype(float)

    # Time handling
    if "time_sec" not in df.columns and "time_min" not in df.columns:
        raise ValueError("Provide either 'time_sec' or 'time_min' in the performance file (use --map if names differ).")

    if "time_sec" not in df.columns:
        df["time_sec"] = pd.to_numeric(df["time_min"], errors="coerce") * 60.0

    if "time_min" not in df.columns:
        df["time_min"] = pd.to_numeric(df["time_sec"], errors="coerce") / 60.0

    # Sort sessions
    df = df.sort_values(["session", "group", "participant_id"])

    # Load perceptions if provided
    percep = None
    if percep_practice:
        p1 = pd.read_csv(percep_practice).rename(columns=maps)
        p1["phase"] = p1.get("phase", "practice")
        percep = p1
    if percep_test:
        p2 = pd.read_csv(percep_test).rename(columns=maps)
        p2["phase"] = p2.get("phase", "test")
        percep = p2 if percep is None else pd.concat([percep, p2], ignore_index=True)

    if percep is not None:
        for c in ["participant_id", "group", "session"]:
            if c in percep.columns:
                if c == "group":
                    percep[c] = percep[c].apply(normalize_group)
                elif c == "session":
                    percep[c] = percep[c].apply(normalize_session)
        # keep only needed columns if present
        keep = [c for c in ["participant_id", "group", "session", "phase", "difficulty", "satisfaction"] if c in percep.columns]
        percep = percep[keep].copy()

    return df, percep

def descriptives(df, outdir):
    # per group-session descriptives
    rows = []
    for (g, s), sub in df.groupby(["group", "session"]):
        n = len(sub)
        # correctness
        if "correctness" in sub.columns:
            p = np.nanmean(sub["correctness"])
            p_m, p_lo, p_hi = ci_prop(p, n)
        else:
            p_m = p_lo = p_hi = np.nan

        # time (median + bootstrap CI), mean CI too
        t = sub["time_min"].to_numpy(dtype=float)
        med, med_lo, med_hi = bootstrap_ci(t, stat_func=np.median, B=2000, alpha=0.05, bca=True)
        mean, mean_lo, mean_hi = ci_mean(t)

        rows.append({
            "group": g, "session": s, "n": n,
            "correct_mean": p_m, "correct_lo": p_lo, "correct_hi": p_hi,
            "time_median_min": med, "time_median_lo": med_lo, "time_median_hi": med_hi,
            "time_mean_min": mean, "time_mean_lo": mean_lo, "time_mean_hi": mean_hi
        })
    out = pd.DataFrame(rows).sort_values(["session","group"])
    out.to_csv(os.path.join(outdir, "table_descriptives_group_session.csv"), index=False)
    return out

def outlier_tables(df, outdir):
    rows = []
    for (g, s), sub in df.groupby(["group","session"]):
        x = sub["time_min"].to_numpy(dtype=float)
        mad_mask = mad_outliers(x, thresh=3.0)
        iqr_mask = iqr_outliers(x, k=1.5)
        rows.append({
            "group": g, "session": s,
            "n": len(x),
            "mad_3x_count": int(mad_mask.sum()),
            "iqr_1p5x_count": int(iqr_mask.sum())
        })
    out = pd.DataFrame(rows).sort_values(["session","group"])
    out.to_csv(os.path.join(outdir, "table_outliers_time.csv"), index=False)
    return out

def pairwise_tests(df, outdir):
    """Pairwise group tests per session:
       - Correctness: difference in proportions (z-test) + Holm
       - Time: log-time OLS pairwise (Welch t-test) + Holm
    """
    # Guard
    df = df.dropna(subset=["group","session"])
    results_corr = []
    results_time = []

    sessions = sorted(df["session"].dropna().unique())
    groups = sorted(df["group"].dropna().unique())

    for s in sessions:
        sub = df[df["session"]==s]
        for (g1, g2) in combinations(groups, 2):
            a = sub[sub["group"]==g1]
            b = sub[sub["group"]==g2]

            # Correctness z-test difference in proportions
            if "correctness" in sub.columns:
                p1 = a["correctness"].mean()
                p2 = b["correctness"].mean()
                n1 = a["correctness"].notna().sum()
                n2 = b["correctness"].notna().sum()
                # Pooled SE
                p_pool = (a["correctness"].sum() + b["correctness"].sum()) / max(1,(n1+n2))
                se = math.sqrt(p_pool*(1-p_pool)*(1/max(1,n1)+1/max(1,n2))) + 1e-12
                z = (p1 - p2) / se
                pval = 2*(1 - stats.norm.cdf(abs(z)))
                results_corr.append({"session": s, "g1": g1, "g2": g2, "p1": p1, "p2": p2, "z": z, "p": pval})

            # Time Welch t-test on log-minutes
            la = np.log(a["time_min"].to_numpy(dtype=float)+1e-9)
            lb = np.log(b["time_min"].to_numpy(dtype=float)+1e-9)
            t, pval = stats.ttest_ind(la, lb, equal_var=False, nan_policy='omit')
            results_time.append({"session": s, "g1": g1, "g2": g2, "t": t, "p": pval})

    dfc = pd.DataFrame(results_corr)
    dft = pd.DataFrame(results_time)

    out_corr = []
    for s, sub in dfc.groupby("session"):
        adj = holm_bonferroni(sub["p"].tolist(), labels=sub.index.tolist(), alpha=0.05)
        merged = sub.copy()
        merged["holm_threshold"] = adj["holm_threshold"].values
        merged["reject_at_alpha"] = adj["reject_at_alpha"].values
        out_corr.append(merged)
    out_corr = pd.concat(out_corr, ignore_index=True) if out_corr else pd.DataFrame()
    out_corr.to_csv(os.path.join(outdir, "table_pairwise_holm_correctness.csv"), index=False)

    out_time = []
    for s, sub in dft.groupby("session"):
        adj = holm_bonferroni(sub["p"].tolist(), labels=sub.index.tolist(), alpha=0.05)
        merged = sub.copy()
        merged["holm_threshold"] = adj["holm_threshold"].values
        merged["reject_at_alpha"] = adj["reject_at_alpha"].values
        out_time.append(merged)
    out_time = pd.concat(out_time, ignore_index=True) if out_time else pd.DataFrame()
    out_time.to_csv(os.path.join(outdir, "table_pairwise_holm_time.csv"), index=False)

    return out_corr, out_time

def glm_models(df, outdir):
    """GLM (binomial) for correctness with cluster-robust SE by participant; snippet fixed effects if available.
       OLS on log-time with cluster-robust SE.
    """
    txt1 = []
    if "correctness" in df.columns:
        f_terms = ["C(group)", "C(session)"]
        if "snippet_id" in df.columns:
            f_terms.append("C(snippet_id)")
        formula = "correctness ~ " + " + ".join(f_terms) + " + C(group):C(session)"
        model = smf.glm(formula, data=df, family=sm.families.Binomial())
        try:
            res = model.fit(cov_type="cluster", cov_kwds={"groups": df["participant_id"]})
        except Exception as e:
            res = model.fit()
        txt1.append(str(res.summary()))
        open(os.path.join(outdir, "table_GLM_correctness_summary.txt"), "w").write("\n".join(txt1))

    # OLS on log-time
    df2 = df.copy()
    df2["log_time_min"] = np.log(df2["time_min"].astype(float) + 1e-9)
    f_terms = ["C(group)", "C(session)"]
    if "snippet_id" in df2.columns:
        f_terms.append("C(snippet_id)")
    formula2 = "log_time_min ~ " + " + ".join(f_terms) + " + C(group):C(session)"
    m2 = smf.ols(formula2, data=df2)
    try:
        r2 = m2.fit(cov_type="cluster", cov_kwds={"groups": df2["participant_id"]})
    except Exception as e:
        r2 = m2.fit()
    open(os.path.join(outdir, "table_OLSlog_time_summary.txt"), "w").write(str(r2.summary()))

def lopo_stability(df, outdir):
    """Leave-one-participant-out for a key contrast: G4 vs G2 (and vs others) on (a) correctness mean diff, (b) time median diff.
       You can extend to more contrasts if needed.
    """
    participants = sorted(df["participant_id"].dropna().unique())
    groups = ["G1","G2","G3","G4"]
    focus_pairs = [("G4","G2"), ("G4","G3"), ("G4","G1")]
    rows = []
    for pid in participants:
        d = df[df["participant_id"]!=pid]
        for s in sorted(d["session"].dropna().unique()):
            sub = d[d["session"]==s]
            for (a,b) in focus_pairs:
                aa = sub[sub["group"]==a]
                bb = sub[sub["group"]==b]
                cm = aa["correctness"].mean() - bb["correctness"].mean()
                tm = np.median(aa["time_min"]) - np.median(bb["time_min"])
                rows.append({"left_out_pid": pid, "session": s, "contrast": f"{a}-{b}", "delta_correct_mean": cm, "delta_time_median_min": tm})
    out = pd.DataFrame(rows)
    out.to_csv(os.path.join(outdir, "table_LOPO_summary.csv"), index=False)
    return out




In [31]:

import pandas as pd
import numpy as np
from pathlib import Path

# Read the raw file defined earlier as PERF_FILE
_raw = pd.read_csv(PERF_FILE)

if "Metrics" in _raw.columns:
    # Identify session columns (everything that's not id/group/metrics)
    meta_cols = ["Metrics", MAPS.get("participant_id", "Participant_ID"), MAPS.get("group", "Group_ID")]
    session_cols = [c for c in _raw.columns if c not in meta_cols]
    
    # Melt to long
    m = _raw.melt(id_vars=meta_cols, value_vars=session_cols, var_name="session", value_name="value")
    
    # Pivot Metrics -> columns
    piv = m.pivot_table(index=[MAPS.get("participant_id", "Participant_ID"), MAPS.get("group", "Group_ID"), "session"],
                        columns="Metrics", values="value", aggfunc="first").reset_index()
    # Flatten columns
    piv.columns.name = None
    
    # Standardize column names expected by analysis
    if "Correctness" in piv.columns:
        piv["correctness"] = pd.to_numeric(piv["Correctness"], errors="coerce")
    else:
        piv["correctness"] = np.nan
    
    # Time assumed to be in MINUTES in this dataset (based on value ranges). Adjust if needed.
    if "Time" in piv.columns:
        piv["time_min"] = pd.to_numeric(piv["Time"], errors="coerce")
        piv["time_sec"] = piv["time_min"] * 60.0
    else:
        piv["time_min"] = np.nan
        piv["time_sec"] = np.nan
    
    # Keep only required columns with standardized names
    piv = piv.rename(columns={MAPS.get("participant_id", "Participant_ID"): "Participant_ID",
                              MAPS.get("group", "Group_ID"): "Group_ID"})
    tidy = piv[["Participant_ID", "Group_ID", "session", "correctness", "time_min", "time_sec"]].copy()
    
    # Save to a temporary prepared CSV and point PERF_FILE to it
    _out_path = Path("perf_prepared.csv")
    tidy.to_csv(_out_path, index=False)
    PERF_FILE = str(_out_path)  # update the path used by subsequent cells
    print("Transformed 'Metrics' file to tidy format at:", _out_path.resolve())
    display(tidy.head(12))
else:
    print("No 'Metrics' column detected — skipping transform.")


Transformed 'Metrics' file to tidy format at: /Users/zhangziyi/Projects/2025-Summer-TOCE-Resubmit-Instruction/perf_prepared.csv


,Participant_ID,Group_ID,session,correctness,time_min,time_sec
0,1,1,P1,0.0,30.42,1825.2
1,1,1,P2,0.0,53.23,3193.8
2,1,1,P3,0.0,30.89,1853.4
3,1,1,P4,0.0,20.31,1218.6
4,1,1,P5,0.0,43.38,2602.8
5,1,1,T0,0.0,44.42,2665.2
6,1,1,T1,0.0,36.58,2194.8
7,1,1,T2,1.0,24.25,1455.0
8,1,1,T3,0.0,25.03,1501.8
9,1,1,T4,0.0,34.46,2067.6


In [32]:
def mde_summary(df, outdir, alpha=0.05, power=0.80):
    """Compute MDE for group contrasts per session:
       - For proportions (correctness): solve for minimal detectable difference under z-test.
       - For means (log-time): solve for minimal detectable difference (Cohen's d equivalent back to minutes via baseline).
    """
    rows = []
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_power = stats.norm.ppf(power)

    sessions = sorted(df["session"].dropna().unique())
    groups = sorted(df["group"].dropna().unique())

    for s in sessions:
        sub = df[df["session"]==s]
        for (g1, g2) in combinations(groups, 2):
            a = sub[sub["group"]==g1]
            b = sub[sub["group"]==g2]

            # proportions: approximate using pbar from pooled
            if "correctness" in sub.columns:
                pbar = pd.concat([a["correctness"], b["correctness"]]).mean()
                n1 = a["correctness"].notna().sum()
                n2 = b["correctness"].notna().sum()
                if n1>0 and n2>0:
                    se = math.sqrt(pbar*(1-pbar)*(1/n1 + 1/n2))
                    mde_p = (z_alpha + z_power) * se
                else:
                    mde_p = np.nan
            else:
                mde_p = np.nan

            # means: on log-time; convert to approx minutes delta at median scale
            la = np.log(a["time_min"].astype(float) + 1e-9)
            lb = np.log(b["time_min"].astype(float) + 1e-9)
            s1 = np.nanstd(la, ddof=1)
            s2 = np.nanstd(lb, ddof=1)
            n1t = la[~np.isnan(la)].size
            n2t = lb[~np.isnan(lb)].size
            sp = np.sqrt(((n1t-1)*s1**2 + (n2t-1)*s2**2) / max(1,(n1t+n2t-2))) if (n1t>1 and n2t>1) else np.nan
            if not np.isnan(sp) and n1t>0 and n2t>0:
                se = sp * math.sqrt(1/n1t + 1/n2t)
                mde_log = (z_alpha + z_power) * se
                # translate to minutes at baseline typical time (geometric mean)
                base = np.exp(np.nanmedian(np.concatenate([la, lb])))
                mde_min = base * (np.exp(mde_log) - 1.0)
            else:
                mde_min = np.nan

            rows.append({"session": s, "g1": g1, "g2": g2, "MDE_correctness_absdiff": mde_p, "MDE_time_minutes": mde_min})
    out = pd.DataFrame(rows)
    out.to_csv(os.path.join(outdir, "table_MDE_summary.csv"), index=False)
    return out


## 4) Load and prepare data

In [33]:
# Returns tidy performance df and optional perceptions df
df, percep = load_and_prepare(PERF_FILE, MAPS, percep_practice=PERCEP_PRACTICE, percep_test=PERCEP_TEST)
print('Prepared df shape:', df.shape)
print(df.head(10))
print('\nPerceptions present:', None if percep is None else percep.shape)

Prepared df shape: (465, 6)
     participant_id  group  session  correctness  time_min  time_sec
5                 1      1      0.0          0.0     44.42    2665.2
49                5      1      0.0          0.0     55.44    3326.4
93                9      1      0.0          0.0     30.47    1828.2
137              13      1      0.0          1.0     40.48    2428.8
213              21      1      0.0          0.0     40.87    2452.2
223              22      1      0.0          0.0     29.70    1782.0
272              27      1      0.0          1.0     15.47     928.2
316              31      1      0.0          1.0     58.75    3525.0
382              41      1      0.0          0.0     54.21    3252.6
426              51      1      0.0          0.0     55.47    3328.2

Perceptions present: None


## 5) Descriptive statistics (group × session)

In [34]:
desc = descriptives(df, OUTDIR)
print('Saved: table_descriptives_group_session.csv')
try:
    import pandas as pd, os
    display(pd.read_csv(os.path.join(OUTDIR, 'table_descriptives_group_session.csv')).head(20))
except Exception as e:
    print('Preview failed:', e)

Saved: table_descriptives_group_session.csv


,group,session,n,correct_mean,correct_lo,correct_hi,time_median_min,time_median_lo,time_median_hi,time_mean_min,time_mean_lo,time_mean_hi
0,1,0.0,11,0.363636,0.079362,0.647911,44.420,29.700000,54.210,42.774545,33.778484,51.770607
1,2,0.0,11,0.363636,0.079362,0.647911,34.400,20.220000,42.230,35.514545,26.470104,44.558987
2,3,0.0,11,0.272727,0.009540,0.535914,37.460,28.210000,45.390,37.707273,31.164354,44.250191
3,4,0.0,10,0.300000,0.015974,0.584026,37.800,28.380000,48.505,39.297000,30.403937,48.190063
4,1,1.0,11,0.181818,0.000000,0.409745,30.335,19.805000,35.210,28.839000,21.722094,35.955906
5,2,1.0,11,0.181818,0.000000,0.409745,30.765,21.835000,36.315,30.779000,20.691888,40.866112
6,3,1.0,11,0.545455,0.251202,0.839707,23.680,16.615000,34.900,27.107000,16.202229,38.011771
7,4,1.0,10,0.800000,0.552082,1.000000,7.810,7.273085,12.850,13.480000,2.695012,24.264988
8,1,2.0,11,0.454545,0.160293,0.748798,20.480,14.360000,24.520,22.539091,16.648185,28.429997
9,2,2.0,11,0.545455,0.251202,0.839707,19.960,11.560000,25.670,23.553636,12.644205,34.463068


## 6) Outlier checks for time (MAD & IQR)

In [35]:
outs = outlier_tables(df, OUTDIR)
print('Saved: table_outliers_time.csv')
try:
    import pandas as pd, os
    display(pd.read_csv(os.path.join(OUTDIR, 'table_outliers_time.csv')).head(20))
except Exception as e:
    print('Preview failed:', e)

Saved: table_outliers_time.csv


,group,session,n,mad_3x_count,iqr_1p5x_count
0,1,0.0,11,0,0
1,2,0.0,11,0,0
2,3,0.0,11,0,0
3,4,0.0,10,0,0
4,1,1.0,11,0,0
5,2,1.0,11,2,1
6,3,1.0,11,1,0
7,4,1.0,10,3,1
8,1,2.0,11,2,1
9,2,2.0,11,1,1


## 7) Pairwise tests with Holm–Bonferroni

In [36]:
pw = pairwise_tests(df, OUTDIR)
print('Saved: table_pairwise_holm_correctness.csv, table_pairwise_holm_time.csv')
try:
    import pandas as pd, os
    print('Correctness (Holm):')
    display(pd.read_csv(os.path.join(OUTDIR, 'table_pairwise_holm_correctness.csv')).head(20))
    print('Time (Holm):')
    display(pd.read_csv(os.path.join(OUTDIR, 'table_pairwise_holm_time.csv')).head(20))
except Exception as e:
    print('Preview failed:', e)

Saved: table_pairwise_holm_correctness.csv, table_pairwise_holm_time.csv
Correctness (Holm):


,session,g1,g2,p1,p2,z,p,holm_threshold,reject_at_alpha
0,0.0,1,2,0.363636,0.363636,0.000000,1.000000,0.008333,False
1,0.0,1,3,0.363636,0.272727,0.457738,0.647141,0.010000,False
2,0.0,1,4,0.363636,0.300000,0.308957,0.757354,0.012500,False
3,0.0,2,3,0.363636,0.272727,0.457738,0.647141,0.016667,False
4,0.0,2,4,0.363636,0.300000,0.308957,0.757354,0.025000,False
5,0.0,3,4,0.272727,0.300000,-0.138170,0.890106,0.050000,False
6,1.0,1,2,0.181818,0.181818,0.000000,1.000000,0.008333,True
7,1.0,1,3,0.181818,0.545455,-1.772811,0.076260,0.010000,True
8,1.0,1,4,0.181818,0.800000,-2.832865,0.004613,0.012500,False
9,1.0,2,3,0.181818,0.545455,-1.772811,0.076260,0.016667,False


Time (Holm):


,session,g1,g2,t,p,holm_threshold,reject_at_alpha
0,0.0,1,2,1.161847,0.258985,0.008333,False
1,0.0,1,3,0.686490,0.501275,0.010000,False
2,0.0,1,4,0.470743,0.643185,0.012500,False
3,0.0,2,3,-0.672448,0.510086,0.016667,False
4,0.0,2,4,-0.734477,0.471640,0.025000,False
5,0.0,3,4,-0.162547,0.872800,0.050000,False
6,1.0,1,2,0.041835,0.967189,0.008333,True
7,1.0,1,3,0.689562,0.501324,0.010000,True
8,1.0,1,4,3.669442,0.003713,0.012500,False
9,1.0,2,3,0.539605,0.596088,0.016667,False


## 8) GLM (binomial) for correctness with cluster-robust SEs

In [37]:
glm = glm_models(df, OUTDIR)
print('Saved: table_GLM_correctness_summary.txt')
try:
    import os
    print(open(os.path.join(OUTDIR, 'table_GLM_correctness_summary.txt'), 'r').read()[:2000])
except Exception as e:
    print('Preview failed:', e)

Saved: table_GLM_correctness_summary.txt
                 Generalized Linear Model Regression Results                  
Dep. Variable:            correctness   No. Observations:                  254
Model:                            GLM   Df Residuals:                      230
Model Family:                Binomial   Df Model:                           23
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -153.92
Date:                Fri, 26 Sep 2025   Deviance:                       307.84
Time:                        13:19:04   Pearson chi2:                     254.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1598
Covariance Type:            nonrobust                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------

## 9) OLS on log-time (cluster-robust SEs)

In [38]:
# OLS summary is included in glm_models(...) output or a separate function in some variants;
# Here we rely on the outputs the script writes.
print('Saved: table_OLSlog_time_summary.txt (if produced by glm_models/OLS step)')
try:
    import os
    print(open(os.path.join(OUTDIR, 'table_OLSlog_time_summary.txt'), 'r').read()[:2000])
except Exception as e:
    print('Preview failed (file may be produced only in certain branches):', e)

Saved: table_OLSlog_time_summary.txt (if produced by glm_models/OLS step)
                            OLS Regression Results                            
Dep. Variable:           log_time_min   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     5.313
Date:                Fri, 26 Sep 2025   Prob (F-statistic):           6.44e-12
Time:                        13:19:04   Log-Likelihood:                -181.36
No. Observations:                 249   AIC:                             410.7
Df Residuals:                     225   BIC:                             495.1
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

## 10) Leave-one-participant-out (LOPO) stability

In [39]:
lopo = lopo_stability(df, OUTDIR)
print('Saved: table_LOPO_summary.csv')
try:
    import pandas as pd, os
    display(pd.read_csv(os.path.join(OUTDIR, 'table_LOPO_summary.csv')).head(20))
except Exception as e:
    print('Preview failed:', e)

Saved: table_LOPO_summary.csv


/Users/zhangziyi/.pyenv/versions/3.10.4/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/zhangziyi/.pyenv/versions/3.10.4/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,left_out_pid,session,contrast,delta_correct_mean,delta_time_median_min
0,1,0.0,G4-G2,NaN,NaN
1,1,0.0,G4-G3,NaN,NaN
2,1,0.0,G4-G1,NaN,NaN
3,1,1.0,G4-G2,NaN,NaN
4,1,1.0,G4-G3,NaN,NaN
5,1,1.0,G4-G1,NaN,NaN
6,1,2.0,G4-G2,NaN,NaN
7,1,2.0,G4-G3,NaN,NaN
8,1,2.0,G4-G1,NaN,NaN
9,1,3.0,G4-G2,NaN,NaN


## 11) Minimum Detectable Effects (MDE) summary

In [40]:
mde = mde_summary(df, OUTDIR)
print('Saved: table_MDE_summary.csv')
try:
    import pandas as pd, os
    display(pd.read_csv(os.path.join(OUTDIR, 'table_MDE_summary.csv')).head(20))
except Exception as e:
    print('Preview failed:', e)

Saved: table_MDE_summary.csv


,session,g1,g2,MDE_correctness_absdiff,MDE_time_minutes
0,0.0,1,2,0.574657,24.785709
1,0.0,1,3,0.556409,20.199641
2,0.0,1,4,0.577047,23.699456
3,0.0,2,3,0.556409,18.920917
4,0.0,2,4,0.577047,22.057500
5,0.0,3,4,0.552992,17.373152
6,1.0,1,2,0.460751,28.041194
7,1.0,1,3,0.574657,26.836842
8,1.0,1,4,0.611356,18.889118
9,1.0,2,3,0.574657,34.204378
